<a href="https://colab.research.google.com/github/s1230038/autoencoder/blob/master/AutoencoderForThesis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!rm *txt *h5 *png *zip
# rm *png *zip *txt

In [6]:
# Simple Autoencoder using Other Loss Function
# Original: https://elix-tech.github.io/ja/2016/07/17/autoencoder.html
# https://colab.research.google.com/drive/1Z_d8APkMUDwXDQIg3OI7E13vH8IZhusM?authuser=1#scrollTo=WmBfOis_mWCH
from keras.layers import Input, Dense
from keras.models import Model
from keras.datasets import mnist
import numpy as np
from keras.datasets import mnist
from keras import backend as K
from keras import metrics
import matplotlib.pyplot as plt
from keras import losses
from tensorflow.python import debug as tf_debug
from itertools import product
from google.colab import files
from keras import optimizers
from keras import constraints as cst

# imititing mean_squared_error():
# 青イルカ P.57では二乗誤差の総和を使用しているが、これをKerasで実装する場合は平均二乗誤差（mean_squared_error)
# をloss に渡す必要がある。なぜなら、Kerasはミニバッチでfit()を計算するからである。青イルカ P.27参照。
# 𝒙 ̂_𝑛 :y_pred,  𝒙_𝑛 : y_true, because x_n is training data which means label.
def i_divergence1(y_true, y_pred):
  y_true = K.clip(y_true, K.epsilon(), 1)
  y_pred = K.clip(y_pred, K.epsilon(), 1)
  return K.mean(y_pred * K.log(y_pred / y_true) - y_pred + y_true, axis=-1)

def i_divergence2(y_true, y_pred):
  y_true = K.clip(y_true, K.epsilon(), 1)
  y_pred = K.clip(y_pred, K.epsilon(), 1)
  return K.mean(y_true * K.log(y_true / y_pred) - y_true + y_pred, axis=-1)

def is_divergence1(y_true, y_pred):
  y_true = K.clip(y_true, K.epsilon(), 1)
  y_pred = K.clip(y_pred, K.epsilon(), 1)
  return K.mean( (y_pred / y_true) - K.log(y_pred / y_true) - 1, axis=-1)

def is_divergence2(y_true, y_pred):
  y_true = K.clip(y_true, K.epsilon(), 1)
  y_pred = K.clip(y_pred, K.epsilon(), 1)
  return K.mean( (y_true / y_pred) - K.log(y_true / y_pred) - 1, axis=-1)

# https://qiita.com/hiroyuki827/items/213146d551a6e2227810
def plot_history_loss(np_loss, np_vloss, name):
    # Plot the loss in the history
    fig, axL = plt.subplots(figsize=(8,6), dpi=500) # グラフの表示準備
    axL.plot(np_loss, label="loss for training")
    axL.plot(np_vloss, label="loss for validation")
    axL.set_title('model loss: ' + name)
    axL.set_xlabel('epoch')
    axL.set_ylabel('loss')
    axL.legend(loc='upper right')
    return fig


epochs = 50
# Download MNIST and standardize, learning
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# we will restrict domain of definition of the input data to the below expression with normalization of the input.
x_train = x_train.astype('float32') / 255. # 画像データは0から1の実数値を取るように規格化
x_test = x_test.astype('float32') / 255.   # {0,1}の二値ではなく実数値であることに注意
# x_trainは (60000, 28, 28) という形をしていますが、784次元の入力になるように (60000, 784) に変形
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
# hyper parameter combination
'''
lossfs = [losses.mean_squared_error, i_divergence1, i_divergence2, is_divergence1, is_divergence2]
acts = ["relu", "sigmoid"]
opzs = ["SGD", "RMSprop", "Adagrad", "Adadelta", "Adam", "Adamax", "Nadam"]
'''
lossfs = [i_divergence1, i_divergence2]
acts = ["relu", "sigmoid"]
opzs = ["SGD", "RMSprop", "Adagrad", "Adadelta", "Adam", "Adamax", "Nadam"]
# ------------------------------------------
for loss, dact, optimizer in product(lossfs, acts, opzs):
  file_prefix = (loss.__name__ + '_' + dact + '_' + optimizer + '_' )
  print("start: " + file_prefix )
  encoding_dim = 32
  input_img = Input(shape=(784,))
  encoded = Dense(encoding_dim, activation='relu', 
                  kernel_constraint=cst.non_neg(), bias_constraint=cst.non_neg())(input_img) 
  decoded = Dense(784, activation=dact, kernel_constraint=cst.non_neg(), bias_constraint=cst.non_neg())(encoded)
  autoencoder = Model(inputs=input_img, outputs=decoded) # https://medium.com/@zhuixiyou/userwarning-update-your-model-call-to-the-keras-2-api-8a6a5955daac
  # autoencoderでは、教師データにラベルを使わないためaccuracyの計算は不要。
  autoencoder.compile(optimizer=optimizer, loss=loss)
  # ------------------------------------------
  fit = autoencoder.fit(x_train, x_train,
                  epochs=epochs,
                  batch_size=256,
                  shuffle=True,
                  verbose=0,
                  validation_data=(x_test, x_test))

  # lossのCSVファイルの保存
  loss_his = fit.history['loss']
  vloss_his = fit.history['val_loss']
  np_loss = np.array(loss_his)
  np_vloss = np.array(vloss_his)
  np.savetxt(file_prefix + "loss_history.txt",     np_loss,  delimiter=",")
  np.savetxt(file_prefix + "val_loss_history.txt", np_vloss, delimiter=",")
  
  # グラフの保存
  fig = plot_history_loss(np_loss, np_vloss, loss.__name__)
  fig.savefig(file_prefix + "loss_history.png")
  plt.close()
  
  # 学習した重みを保存
  autoencoder.save_weights(file_prefix + 'autoencoder.h5')
  
  autoencoder.reset_states()

start: i_divergence1_relu_SGD_


KeyboardInterrupt: ignored

In [7]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

# ------------------------------------------
for loss, dact, optimizer in product(lossfs, acts, opzs):
  file_prefix = (loss.__name__ + '_' + dact + '_' + optimizer + '_' )
  print("start: " + file_prefix )
  encoding_dim = 32
  input_img = Input(shape=(784,))
  encoded = Dense(encoding_dim, activation='relu', 
                  kernel_constraint=cst.non_neg(), bias_constraint=cst.non_neg())(input_img) 
  decoded = Dense(784, activation=dact, kernel_constraint=cst.non_neg(), bias_constraint=cst.non_neg())(encoded)
  autoencoder = Model(inputs=input_img, outputs=decoded) # https://medium.com/@zhuixiyou/userwarning-update-your-model-call-to-the-keras-2-api-8a6a5955daac
  # autoencoderでは、教師データにラベルを使わないためaccuracyの計算は不要。
  autoencoder.compile(optimizer=optimizer, loss=loss)
  # ------------------------------------------
  # 保存した重みを読み込み
  autoencoder.load_weights(file_prefix + 'autoencoder.h5')
  decoded_imgs = autoencoder.predict(x_test)
  # 0-9を表示する
  plt.figure(figsize=(20, 4))
  label = 0
  for i in range(1000):
      if label > 10:
          break
      elif label != y_test[i]:
          continue
      # オリジナルのテスト画像を表示
      ax = plt.subplot(2, 10, label+1)
      plt.imshow(x_test[i].reshape(28, 28))
      plt.gray()
      ax.get_xaxis().set_visible(False)
      ax.get_yaxis().set_visible(False)

      # 変換された画像を表示
      ax = plt.subplot(2, 10, label+1+10)
      plt.imshow(decoded_imgs[i].reshape(28, 28))
      plt.gray()
      ax.get_xaxis().set_visible(False)
      ax.get_yaxis().set_visible(False)
      
      label+=1
  #  plt.show() #show()するとファイルに保存できないことに注意。
  plt.savefig(file_prefix + 'numbers.png')
  autoencoder.reset_states()

start: i_divergence1_relu_SGD_


/usr/local/lib/python3.6/dist-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


start: i_divergence1_relu_RMSprop_
start: i_divergence1_relu_Adagrad_
start: i_divergence1_relu_Adadelta_
start: i_divergence1_relu_Adam_
start: i_divergence1_relu_Adamax_
start: i_divergence1_relu_Nadam_
start: i_divergence1_sigmoid_SGD_
start: i_divergence1_sigmoid_RMSprop_
start: i_divergence1_sigmoid_Adagrad_
start: i_divergence1_sigmoid_Adadelta_
start: i_divergence1_sigmoid_Adam_
start: i_divergence1_sigmoid_Adamax_
start: i_divergence1_sigmoid_Nadam_
start: i_divergence2_relu_SGD_
start: i_divergence2_relu_RMSprop_
start: i_divergence2_relu_Adagrad_
start: i_divergence2_relu_Adadelta_
start: i_divergence2_relu_Adam_
start: i_divergence2_relu_Adamax_
start: i_divergence2_relu_Nadam_
start: i_divergence2_sigmoid_SGD_
start: i_divergence2_sigmoid_RMSprop_
start: i_divergence2_sigmoid_Adagrad_
start: i_divergence2_sigmoid_Adadelta_
start: i_divergence2_sigmoid_Adam_
start: i_divergence2_sigmoid_Adamax_
start: i_divergence2_sigmoid_Nadam_


In [0]:
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import tensorflow as tf
from skimage.measure import compare_ssim, compare_psnr
import cv2

# ------------------------------------------
for loss, dact, optimizer in product(lossfs, acts, opzs):
  file_prefix = (loss.__name__ + '_' + dact + '_' + optimizer + '_' )
  print("start: " + file_prefix )
  encoding_dim = 32
  input_img = Input(shape=(784,))
  encoded = Dense(encoding_dim, activation='relu', 
                  kernel_constraint=cst.non_neg(), bias_constraint=cst.non_neg())(input_img) 
  decoded = Dense(784, activation=dact, kernel_constraint=cst.non_neg(), bias_constraint=cst.non_neg())(encoded)
  autoencoder = Model(inputs=input_img, outputs=decoded) # https://medium.com/@zhuixiyou/userwarning-update-your-model-call-to-the-keras-2-api-8a6a5955daac
  # autoencoderでは、教師データにラベルを使わないためaccuracyの計算は不要。
  autoencoder.compile(optimizer=optimizer, loss=loss)
  # ------------------------------------------
  # 保存した重みを読み込み
  autoencoder.load_weights(file_prefix + 'autoencoder.h5')
  decoded_imgs = autoencoder.predict(x_test)
  # 0-9を表示する
  plt.figure(figsize=(20, 4))
  plt.grid(False)
  plt.axis('off')
  label = 0
  for i in range(1000):
      if label > 10:
          break
      elif label != y_test[i]:
          continue

      xfname = str(label) + '_x_test_numbers.png'
      yfname = file_prefix + str(label) + '_decoded_numbers.png'
      # オリジナルのテスト画像を表示
      plt.imshow(x_test[i].reshape(28, 28))
      plt.gray()
      plt.savefig(xfname)

      # 変換された画像を表示
      plt.imshow(decoded_imgs[i].reshape(28, 28))
      plt.gray()
      plt.savefig(yfname)
      
      # https://qiita.com/redshoga/items/5e36d784a322c940f38e
      # http://scikit-image.org/docs/dev/api/skimage.measure.html#skimage.measure.compare_ssim
      img1 = cv2.imread(xfname, cv2.IMREAD_GRAYSCALE)
      img2 = cv2.imread(yfname, cv2.IMREAD_GRAYSCALE)
      val = compare_ssim(img1, img2)
      
      sfile = open('ssim.txt' ,mode='a')
      sfile.write( xfname + ' - ' + yfname + ': ' + str(val) + '\n')
      sfile.close()
      
      label+=1
  #  plt.show() #show()するとファイルに保存できないことに注意。
  autoencoder.reset_states()

start: i_divergence1_relu_SGD_


/usr/local/lib/python3.6/dist-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


start: i_divergence1_relu_RMSprop_
start: i_divergence1_relu_Adagrad_
start: i_divergence1_relu_Adadelta_
start: i_divergence1_relu_Adam_
start: i_divergence1_relu_Adamax_
start: i_divergence1_relu_Nadam_
start: i_divergence1_sigmoid_SGD_
start: i_divergence1_sigmoid_RMSprop_
start: i_divergence1_sigmoid_Adagrad_
start: i_divergence1_sigmoid_Adadelta_
start: i_divergence1_sigmoid_Adam_
start: i_divergence1_sigmoid_Adamax_
start: i_divergence1_sigmoid_Nadam_


In [0]:
# 保存した重みのダウンロード
from google.colab import files
!zip -r h5.zip *.h5
!zip -r png.zip *.png
!zip -r txt.zip *.txt
files.download('h5.zip')
files.download('png.zip')
files.download('txt.zip')